### Conversation Q&A Chatbot
In many Q&A applications we want to allow the user to have a back-and-forth conversation, meaning the application needs some sort of "memory" of past questions and answers, and some logic for incorporating those into its current thinking.

In this we focus on adding logic for incorporating historical messages. Further details on chat history management is covered in the previous videos.

In [1]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model='llama-3.3-70b-versatile')

In [2]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
webLoadder = WebBaseLoader(web_path='https://en.wikipedia.org/wiki/Greek_mythology')
data = webLoadder.load()
data

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Greek_mythology', 'title': 'Greek mythology - Wikipedia', 'language': 'en'}, page_content='\n\n\n\nGreek mythology - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n

In [4]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splitted_data = splitter.split_documents(data)
splitted_data

[Document(metadata={'source': 'https://en.wikipedia.org/wiki/Greek_mythology', 'title': 'Greek mythology - Wikipedia', 'language': 'en'}, page_content='Greek mythology - Wikipedia\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nJump to content\n\n\n\n\n\n\n\nMain menu\n\n\n\n\n\nMain menu\nmove to sidebar\nhide\n\n\n\n\t\tNavigation\n\t\n\n\nMain pageContentsCurrent eventsRandom articleAbout WikipediaContact us\n\n\n\n\n\n\t\tContribute\n\t\n\n\nHelpLearn to editCommunity portalRecent changesUpload fileSpecial pages\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nAppearance\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nDonate\n\nCreate account\n\nLog in\n\n\n\n\n\n\n\n\nPersonal tools\n\n\n\n\n\nDonate Create account Log in\n\n\n\n\n\n\t\tPages for logged out editors learn more\n\n\n\nContributionsTalk\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nContents\nmove to sidebar\nhide\n\n\n\n\n(Top)\n\n\n\n\n\n

In [5]:
embedding = OllamaEmbeddings(model='embeddinggemma:300m')
vectordb = Chroma.from_documents(documents =splitted_data ,embedding = embedding)
vectordb

In [6]:
retriver = vectordb.as_retriever()
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ('system',system_prompt),
    ('human','{input}')
])

In [7]:
QAndAChain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retriver,QAndAChain)

In [10]:
response = rag_chain.invoke({'input':'what is Greek pantheon?'})
response['answer']

"The Greek pantheon refers to the collection of gods and goddesses in Greek mythology. It includes a wide range of deities, such as Zeus, Athena, and Apollo, who were believed to have control over various aspects of life and the universe. I don't know the full extent of the pantheon from the given context, but it is a central part of Greek mythology."

### Adding Chat History

In [21]:
from langchain.chains import create_history_aware_retriever
from langchain.prompts import MessagesPlaceholder

contextulized_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextulized_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [22]:
history_aware_retriver = create_history_aware_retriever(llm,retriver,contextualize_q_prompt)
history_aware_retriver

RunnableBinding(bound=RunnableBranch(branches=[(RunnableLambda(lambda x: not x.get('chat_history', False)), RunnableLambda(lambda x: x['input'])
| VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x113d60490>, search_kwargs={}))], default=ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], t

In [24]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [25]:
QAndAChain = create_stuff_documents_chain(llm,qa_prompt)
context_rag_chain = create_retrieval_chain(history_aware_retriver,QAndAChain)

In [26]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []
question = 'what is Greek pantheon?'
response1 = context_rag_chain.invoke({
    'input':question,
    'chat_history':chat_history
})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=response1['answer'])
    ]
)

another_question = 'Explain more about it'

response2 = context_rag_chain.invoke({
    'input':another_question,
    'chat_history':chat_history
})
response2['answer']

'The Greek pantheon refers to the collection of gods and goddesses in Greek mythology, which includes the Twelve Olympians and other deities. The context mentions that further information can be found in "Ancient Greek religion", "Twelve Olympians", and "List of Greek deities", but it does not provide a detailed explanation itself. The Greek pantheon is a complex system of gods and goddesses, each with their own powers and roles, such as Zeus, who is mentioned in the context as disguising himself as a swan.'

In [27]:
chat_history

[HumanMessage(content='what is Greek pantheon?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I don't know the specifics of the Greek pantheon from the provided context, but it's generally understood to refer to the collection of gods and goddesses in Greek mythology. The context provided doesn't offer a detailed explanation of the Greek pantheon. It mentions Greek mythology, but doesn't elaborate on the pantheon itself.", additional_kwargs={}, response_metadata={})]

In [29]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

session = {}

def get_session(session_id:str) ->BaseChatMessageHistory:
    if session_id not in session:
        session[session_id] = ChatMessageHistory()
    return session[session_id]

contextual_rag_chain = RunnableWithMessageHistory(
    context_rag_chain,
    get_session,
    input_messages_keys='input',
    history_messages_key='chat_history'
)

In [30]:
config = {
    'configurable':{
        'session_id':'abc123'
    }
}
response = contextual_rag_chain.invoke({'input':'what is explained about age of gods and mortals'},config = config)
response

Error in RootListenersTracer.on_chain_end callback: KeyError('output')


{'input': 'what is explained about age of gods and mortals',
 'chat_history': [],
 'context': [Document(id='d9fb5839-f609-4ae1-99c5-20275b6d2d87', metadata={'title': 'Greek mythology - Wikipedia', 'source': 'https://en.wikipedia.org/wiki/Greek_mythology', 'language': 'en'}, page_content='Secondary sources'),
  Document(id='a49df7c5-0a27-46e4-9065-71c2cc194ea1', metadata={'source': 'https://en.wikipedia.org/wiki/Greek_mythology', 'title': 'Greek mythology - Wikipedia', 'language': 'en'}, page_content='Further reading\n\nAncient Greece portalMyths portalReligion portalMythology portalHistory portal'),
  Document(id='ac9d2fb6-8d41-49e4-a926-fea52ad7da9b', metadata={'title': 'Greek mythology - Wikipedia', 'source': 'https://en.wikipedia.org/wiki/Greek_mythology', 'language': 'en'}, page_content='Origin theories\nSee also: Mycenaean religion; Mycenaean deities; and Similarities between Roman, Greek and Etruscan mythologies'),
  Document(id='dbdb75a7-bebb-4e24-87bf-ce57d0d73aa6', metadata={'